In [908]:
import pygame,sys
from pygame.locals import *
from random import randint
import math
import numpy as np

In [909]:
#colors
WHITE = (255, 255, 255)
BLACK=(0,0,0)

In [910]:
def desloc(x,y,ori):
    des=1
    if(ori=='r'):
        return (x+des,y)
    elif(ori=='l'):
        return (x-des,y)
    elif(ori=='u'):
        return (x,y-des)
    elif (ori=='d'):
        return (x,y+des)

In [911]:
def oriNumb(ori):
    
    if(ori=='r'):
        return 0
    elif(ori=='d'):
        return 1
    elif(ori=='l'):
        return 2
    elif(ori=='u'):
        return 3

In [912]:
class food(pygame.sprite.Sprite):
    
    def __init__(self,x,y,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=self.image.get_rect()
        self.rect.x=x
        self.rect.y=y

In [913]:
#head
class head(pygame.sprite.Sprite):
    
    def __init__(self,x,y,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=pygame.rect.Rect(x,y,int(img.get_width()*0.75),int(img.get_height()+0.75))
        self.orientation=''
        self.turns=[]
    
    def get_render_rect(self):
        
        return ((self.rect.x-int(self.image.get_width()*0.25)),(self.rect.y-int(self.image.get_height()*0.25)))

In [914]:
#body
class body(pygame.sprite.Sprite):
    def __init__(self,x,y,ori,img):
        
        pygame.sprite.Sprite.__init__(self)
        self.image=img
        self.rect=pygame.rect.Rect(x,y,int(img.get_width()*0.75),int(img.get_height()*0.75))
        self.orientation=ori
        self.turns=[]
    
    def move(self):
        
        if(len(self.turns)==0):
            self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,self.orientation)
            
        else:
            turnx,turny,turno= self.turns[0]
            virtualx,virtualy=desloc(self.rect.x,self.rect.y,self.orientation)
            
            if(((self.orientation=='r' and virtualx>turnx) or (self.orientation=='l' and virtualx<turnx) 
               or (self.orientation=='u' and virtualy<turny) or (self.orientation=='d' and virtualy>turny))
              and self.orientation!=turno):
                
                self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,turno)
                self.orientation=turno
                self.turns.pop(0)
                
            else:
                self.rect.x,self.rect.y=desloc(self.rect.x,self.rect.y,self.orientation)

    def get_render_rect(self):
        
        return ((self.rect.x-int(self.image.get_width()*0.25)),(self.rect.y-int(self.image.get_height()*0.25)))
                

In [915]:
#Snake
class Snake:
    
    def __init__(self,x,y,headIm,ori='r'):
        
        self.head=head(x,y,headIm)
        self.head.orientation=ori
        self.blocks=[]
        self.orientation=ori   
        self.headImg=headIm
        self.bodyImg=pygame.Surface((0,0))
        self.walls=[]
        
    def move(self,ori):
        
        if(ori!=self.orientation):
            #turn happened
            turn=(self.head.rect.x,self.head.rect.y,ori)
            for block in self.blocks:
                block.turns.append(turn)
            self.orientation=ori
            self.head.orientation=ori
   
        self.head.rect.x,self.head.rect.y=desloc(self.head.rect.x,self.head.rect.y,ori)
        
        for b in self.blocks:
            b.move()

    def createWalls(self,dim):
        
        r1=pygame.rect.Rect(0,-1,dim,1)
        r2=pygame.rect.Rect(-1,0,1,dim)
        r3=pygame.rect.Rect(dim,0,1,dim)
        r4=pygame.rect.Rect(0,dim,dim,1)
        self.walls.append(r1)
        self.walls.append(r2)
        self.walls.append(r3)
        self.walls.append(r4)
    
    def isAlive(self):
        
        head_rect=self.head.rect
        
        for bloc in self.blocks:
            if(head_rect.colliderect(bloc.rect)):
                return False
        
        for wall in self.walls:
            if(head_rect.colliderect(wall)):
                return False
        return True
    
    def calcBodyPosition(self,bloc):
        
        ori=bloc.orientation
        
        #values
        spacing=int(0.70*bloc.image.get_width())
        dist=spacing+bloc.image.get_width()
        
        #virtual values
        virtualx,virtualy=0,0
        
        if(ori=='r'):
            virtualx,virtualy=(bloc.rect.x-dist),(bloc.rect.y)

        elif(ori=='l'):
            virtualx,virtualy=(bloc.rect.x+dist),(bloc.rect.y)

        elif(ori=='u'):
            virtualx,virtualy=(bloc.rect.x),(bloc.rect.y+dist)

        elif(ori=='d'):
            virtualx,virtualy=(bloc.rect.x),(bloc.rect.y-dist)
        
        return (virtualx,virtualy)
    
    def addBlock(self):
        
        lastbloc=self.head
        
        if(len(self.blocks)>0):
            lastbloc=self.blocks[-1]
        bx,by=self.calcBodyPosition(lastbloc)
        nbody= body(bx,by,lastbloc.orientation,lastbloc.image)
        nbody.turns=list(lastbloc.turns)
        self.blocks.append(nbody)
        
    def eaten(self,food):
        
        if(self.head.rect.colliderect(food.rect)):
            return True
        return False
    
    
    def obstDist(self,ori):
        
        dx,dy=0,0
        if(ori==0):
            dx=1
        elif(ori==2):
            dx=-1
        elif(ori==3):
            dy=-1
        elif(ori==1):
            dy=1
        
        virtualx,virtualy=int(self.head.rect.x+(self.head.image.get_width()*0.5)),int(self.head.rect.y+(self.head.image.get_height()*0.5))
        flag=True
        while(flag):
            virtualx+=dx
            virtualy+=dy
            for bloc in self.blocks:
                bx,by=bloc.get_render_rect()
                sp=int(bloc.image.get_width()*0.7)
                nrect=pygame.rect.Rect(bx,by,(bloc.image.get_width()+sp),(bloc.image.get_height()+sp))
                if(nrect.collidepoint(virtualx,virtualy)):
                    flag=False
                    break
            
            if(flag):    
                for wall in self.walls:
                    if(wall.collidepoint(virtualx,virtualy)):
                        flag=False
                        break
        if(ori==0 or ori==2):
            return abs((self.head.rect.x-virtualx))
        elif(ori==1 or ori==3):
            return abs((self.head.rect.y-virtualy))

    
    def getAtr(self):
        
        nori=oriNumb(self.orientation)
        return [self.obstDist((nori+3)%4),self.obstDist(nori),self.obstDist((nori+1)%4)]

In [916]:
def renderSnake(display,snake):
    
    headImg=snake.headImg
    bodyImg=snake.bodyImg
    
    #add head
    display.blit(headImg,snake.head.get_render_rect())
    
    #add body
    for body in snake.blocks:
        display.blit(bodyImg,body.get_render_rect()) 
        

In [917]:
def renderFood(display,food):
    
    display.blit(food.image,food.rect)

In [918]:
def newFood(foodImg,dim):
    
    fos=5
    return food(randint(fos,(dim-foodImg.get_width()-fos)),randint(fos,(dim-foodImg.get_height()-fos)),foodImg)

In [ ]:
def foodPos(snake,food):
    
    hip=math.sqrt((snake.head.rect.x-food.rect.x)**2+(snake.head.rect.y-food.rect.y)**2)
    cat=0
    if(snake.orientation=='r'):
        cat=snake.head.rect.x
        if(food.rect.y>=snake.head.rect.y):
            return (360-math.degrees(np.arccos(cat/hip)))
        else:
            return math.degrees(np.arccos(cat/hip))

    elif(snake.orientation=='l'):
        cat=abs(snake.head.rect.x-food.rect.x)
        if(food.rect.y<=snake.head.rect.y):
            return (360-math.degrees(np.arccos(cat/hip)))
        else:
            return math.degrees(np.arccos(cat/hip))
    
    elif(snake.orientation=='u'):
        cat=snake.head.rect.y-food.rect.y
        if(food.rect.x<=snake.head.rect.x):
            return (360-math.degrees(np.arccos(cat/hip)))
        else:
            return math.degrees(np.arccos(cat/hip))
    
    elif(snake.orientation=='d'):
        cat=food.rect.y-snake.head.rect.y
        if(food.rect.x>=snake.head.rect.x):
            return (360-math.degrees(np.arccos(cat/hip)))
        else:
            return math.degrees(np.arccos(cat/hip))
     

In [ ]:
#init
pygame.init()
dim=300
displaysurf=pygame.display.set_mode((dim,dim))
pygame.display.set_caption('Snaike')
fpsClock = pygame.time.Clock()
FPS = 30


#emilia variables
emiliaImg = pygame.image.load('blue.png')
emiliaSnake = Snake(int(dim/2)+100,int(dim/2)-100,emiliaImg,ori='d')
emiliaSnake.createWalls(dim)

#subaro
subaroImg=pygame.image.load('blue.png')
emiliaSnake.bodyImg=subaroImg

#game over
gameoverImg=pygame.image.load('gameover.jpg')

#food
foodImg=pygame.image.load('hamb.png')
foood=food(dim/2,dim/2,foodImg)

#emiliaSnake.addBlock()
orientation=emiliaSnake.orientation
gameOverFlag=False

#game loop
while True:   
    displaysurf.fill(WHITE)
    
    for event in pygame.event.get():
        if event.type== QUIT:
            pygame.quit()
            sys.exit()
        elif event.type==KEYDOWN:
            if event.key==K_UP and orientation!='d':
                orientation='u'
            elif event.key==K_DOWN and orientation!='u':
                orientation='d'
            elif event.key==K_RIGHT and orientation!='l':
                orientation='r'
            elif event.key==K_LEFT and orientation!='r':
                orientation='l'
            elif event.key==K_f:
                emiliaSnake = Snake(int(dim/2),int(dim/2),emiliaImg)
                emiliaSnake.createWalls(dim)
                emiliaSnake.bodyImg=subaroImg
                gameOverFlag=False
            elif event.key==K_n:
                pygame.quit()
                sys.exit()
    
    #emilia motion
    emiliaSnake.move(orientation)
    
    if(not emiliaSnake.isAlive()):
        gameOverFlag=True
        
        displaysurf.blit(gameoverImg, (int((dim-gameoverImg.get_width())/2),(int((dim-gameoverImg.get_height())/2)-50)))
        pygame.display.update()

    if(emiliaSnake.eaten(foood)):
        
        foood=newFood(foodImg,dim)
        emiliaSnake.addBlock()
    
    if(not gameOverFlag):
        renderSnake(displaysurf,emiliaSnake)
        renderFood(displaysurf,foood)
        print(foodPos(emiliaSnake,foood))
        pygame.display.update()
        
    fpsClock.tick(FPS)

            

45.2879160665571
45.57872556560777
45.87245712290204
46.16913932790742
46.46880071438583
46.771469740034085
47.07717476516281
47.38594403038882
47.69780563331631
48.01278750418334
48.33091738045006
48.65222278030633
48.976730975077565
49.30446896050799
49.63546342690264
49.96974072811031
50.30732684933118
50.64824737373527
50.99252744787896
51.34019174590991
51.69126443255228
52.04576912486749
52.403728852788014
52.76516601842533
53.13010235415599
53.49855887949368
53.87055585675915
54.24611274556325
54.625248156123924
55.00797980144134
55.39432444836144
55.7842978675626
56.17791478250605
56.5751888173962
56.976132444203365
57.38075692880717
57.789072276326245
58.201087175705574
58.616808943640976
59.03624346792648
59.4593951503185
59.88626684901757
60.316859820876694
60.751173663453024
61.18920625702693
61.63095370672142
62.076410284861616
62.52556837372287
62.97841840882297
63.43494882292201
63.8951459909007
64.35899417569472
64.82647547546983
65.2975697722287
65.77225468204583
66.25